In this jupyter notebook we try to implement the Active Learning algorithm to find self assembling tripeptides. We start out by importing important libraries for our algorithm.

In [49]:
#importing stuff
import numpy as np                                      
import pandas as pd
import matplotlib.pyplot as plt
import math,os,random,sys,h5py
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer

The function below takes a pandas dataframe and normalizes it. 

In [50]:
def normalize_pandas_data(data):
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
    new_data = min_max_scaler.fit_transform(data)
    return pd.DataFrame(new_data, index=data.index, columns=data.columns)

Next we read the data from the files to use it. Since we have AP data for all peptides we use this data instead of running a simulation in each step.
Note: 'sap' is a list of all self-assembling peptides as predicted by Tuttle et al.3

In [51]:
APs = pd.read_csv("tripeptide_AP.txt", sep=": ", index_col=0, header=None, engine = 'python')              #AP scores from simulation
f = h5py.File('tripeptides.hdf5','r')                                                                      #reading data from the file generated from judred generator
peps = np.array(f.get('peptides'))
feas = np.array(f.get('features'))
data = np.array(f.get('data'))
f.close()
judp = normalize_pandas_data(pd.DataFrame(data, columns = feas, index = peps))                              #arranging it into a pandas data frame
smiles = pd.read_csv("smi.txt", sep=": ", engine='python')                                                  #smiles data of peptides by their names
modp = normalize_pandas_data(pd.read_csv('Mordred_Parameters_to_be_used.csv', sep=',', engine='python'))    #mordred data in same order as the smiles data
sap = ['PHE-PHE-PHE','TRP-PHE-PHE','PHE-TRP-PHE','PHE-PHE-TRP','ILE-PHE-TRP','PHE-TYR-ILE','PRO-TRP-PHE','TRP-PHE-LEU','ILE-PHE-PHE','VAL-PHE-TRP','PRO-PHE-PHE','PHE-PHE-MET','TRP-PHE-PHE','VAL-PHE-PHE','MET-PHE-PHE','TRP-LEU-LEU','SER-PHE-TRP','ILE-MET-TRP','LEU-CYS-PHE','SER-SER-PHE','SER-CYS-TRP','LYS-TRP-PHE','LYE-PHE-TRP','LYS-TRP-ASP','LYS-HSE-ASP','TRP-LYS-ASP','HSE-LYS-ASP','LYS-TYR-ASP','LYS-PHE-ASP','LYS-TRP-GLU','TRP-LYS-GLU','LYS-GLU-HSE','LYS-TYR-GLU','LYS-HSE-GLU','PRO-CYS-PHE','THR-SER-PHE','GLY-PHE-PHE','VAL-ALA-TRP']       #list of self assenbling peptides

In [52]:
APs

,1
0,
ALA-ALA-ALA,1.648900
ASN-GLN-CYS,1.739705
ALA-ALA-CYS,2.176944
PRO-VAL-ALA,1.126057
VAL-PRO-ALA,1.095680
...,...
VAL-SER-ALA,1.381075
VAL-SER-GLN,1.664895
VAL-SER-GLU,1.168641


In [53]:
judp

,b'SP2',b'NH2',b'MW',b'S',b'LogP WW',b'Z',b'MaxASA',b'RotRatio',b'Bulkiness',b'OH'
b'ALA-ALA-CYS',-1.000000,-1.0,-0.617271,-0.333333,0.314700,0.000000,-0.583794,-1.000000,-0.041781,-1.000000
b'ALA-ALA-ASP',-0.916667,-1.0,-0.555590,-1.000000,-0.138716,-0.333333,-0.488029,-0.916667,-0.106732,-1.000000
b'ALA-ALA-GLU',-0.916667,-1.0,-0.483225,-1.000000,0.026915,-0.333333,-0.377532,-0.937500,-0.037767,-1.000000
b'ALA-ALA-PHE',-0.500000,-1.0,-0.390007,-1.000000,0.480331,0.000000,-0.314917,-0.500000,0.189564,-1.000000
b'ALA-ALA-GLY',-1.000000,-1.0,-0.855167,-1.000000,0.124224,0.000000,-0.815838,-1.000000,-0.408867,-1.000000
...,...,...,...,...,...,...,...,...,...,...
b'TYR-TYR-THR',0.000000,-1.0,0.322855,-1.000000,0.378882,0.000000,0.421731,-0.250000,0.519066,1.000000
b'TYR-TYR-VAL',0.000000,-1.0,0.312687,-1.000000,0.511387,0.000000,0.429098,-0.400000,0.730706,0.333333
b'TYR-TYR-TRP',0.666667,-1.0,0.762155,-1.000000,0.451346,0.000000,0.837937,0.666667,0.734355,0.333333
b'TYR-TYR-TYR',0.500000,-1.0,0.643233,-1.000000,0.354037,0.000000,0.756906,0.500000,0.601533,1.000000


In [54]:
smiles

,Peptides,smiles
0,ALA-ALA-ALA,[H]N[C@@H](C)C(=O)N[C@@H](C)C(=O)N[C@@H](C)C(O)=O
1,ALA-ALA-CYS,[H]N[C@@H](C)C(=O)N[C@@H](C)C(=O)N[C@@H](CS)C(...
2,ALA-ALA-ASP,[H]N[C@@H](C)C(=O)N[C@@H](C)C(=O)N[C@@H](CC(O)...
3,ALA-ALA-GLU,[H]N[C@@H](C)C(=O)N[C@@H](C)C(=O)N[C@@H](CCC(O...
4,ALA-ALA-PHE,[H]N[C@@H](C)C(=O)N[C@@H](C)C(=O)N[C@@H](CC1=C...
...,...,...
7995,TYR-TYR-SER,[H]N[C@@H](CC1=CC=C(O)C=C1)C(=O)N[C@@H](CC1=CC...
7996,TYR-TYR-THR,[H]N[C@@H](CC1=CC=C(O)C=C1)C(=O)N[C@@H](CC1=CC...
7997,TYR-TYR-VAL,[H]N[C@@H](CC1=CC=C(O)C=C1)C(=O)N[C@@H](CC1=CC...
7998,TYR-TYR-TRP,[H]N[C@@H](CC1=CC=C(O)C=C1)C(=O)N[C@@H](CC1=CC...


In [55]:
modp

,Unnamed: 0,AATS0s,AATS2s,AATSC0are,AATSC0s,AATSC0se,AATSC1are,AATSC1pe,AATSC1v,AATSC2s,...,NsNH2,NsssN,RotRatio,SaasC,ETA_dEpsilon_B,ETA_shape_p,ETA_shape_y,Kier3,SpMAD_A,SlogP_VSA4
0,-1.00000,-0.232323,-0.252278,0.023694,-0.020400,-0.042373,0.242472,0.224388,-0.775307,0.081151,...,-0.5,-1.0,-0.174312,-1.000000,-1.000000,0.938632,0.615206,-0.594865,-0.999740,-1.000000
1,-0.99975,-0.296009,-0.297832,-0.048211,-0.062218,-0.096375,0.298121,0.236236,-0.818339,0.054558,...,-0.5,-1.0,-0.009174,-1.000000,-1.000000,0.906403,0.232977,-0.580930,-0.753054,-1.000000
2,-0.99950,0.246914,0.224292,0.462118,0.419799,0.416672,-0.190430,-0.202021,-0.248698,0.481399,...,-0.5,-1.0,0.045872,-1.000000,-1.000000,0.708948,0.617650,-0.387127,-1.000000,-1.000000
3,-0.99925,0.077482,0.044914,0.321717,0.288771,0.280594,0.010702,-0.012467,-0.337247,0.437974,...,-0.5,-1.0,0.173346,-1.000000,-1.000000,0.587540,0.410140,-0.283539,-0.814009,-1.000000
4,-0.99900,-0.563853,-0.451082,-0.454447,-0.415300,-0.467848,0.590004,0.540307,-0.029802,-0.190107,...,-0.5,-1.0,-0.234362,-0.368639,0.015732,-0.015586,-0.042109,-0.460807,-0.194774,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0.99900,-0.346334,-0.291621,-0.164071,-0.244333,-0.139386,0.167825,0.094548,0.900114,-0.468058,...,-0.5,-1.0,-0.256881,0.110087,0.631559,-0.411206,0.020975,-0.207683,0.288059,-1.000000
7996,0.99925,-0.422228,-0.347084,-0.239328,-0.309133,-0.215300,0.261171,0.187260,0.798861,-0.377276,...,-0.5,-1.0,-0.294412,0.106351,0.514444,-0.270064,0.208313,-0.176539,0.145550,-1.000000
7997,0.99950,-0.620089,-0.523298,-0.487674,-0.495633,-0.465288,0.519848,0.448797,0.525072,-0.279930,...,-0.5,-1.0,-0.294412,0.224857,0.368261,-0.225151,0.172017,-0.176539,0.145550,-0.333333
7998,0.99975,-0.708493,-0.501028,-0.642080,-0.655456,-0.613518,0.537129,0.468147,0.898086,-0.423522,...,-0.5,-1.0,-0.457405,0.770275,0.938094,-0.727992,0.044843,-0.195957,0.644368,-1.000000


In [56]:
x_train_jd = [np.array(judp.loc[b'ALA-ALA-ALA'])]                  # initializing the training data with data of polyalanine 
x_train_md = [np.array(modp.iloc[1])]
Y_train = np.array(APs.loc['ALA-ALA-ALA'])
print(x_train_jd)
print(x_train_md)
print(Y_train)

[array([-1.        , -1.        , -0.7827501 , -1.        ,  0.29192555,
        0.        , -0.7237569 , -1.        , -0.11330056, -1.        ],
      dtype=float32)]
[array([-0.99974997, -0.29600933, -0.29783152, -0.04821083, -0.06221802,
       -0.09637477,  0.29812115,  0.23623551, -0.81833914,  0.05455816,
       -0.0476218 , -0.28325474, -0.38545507,  0.49770944,  0.42685493,
        0.51218476, -0.18984381, -0.27525841, -0.23278923, -0.23719899,
        0.24805536,  0.58168666, -0.17788701,  0.42878925, -0.85577319,
       -0.12828389,  0.12129519,  0.1910701 , -0.4782197 , -0.80906926,
        0.59273196,  0.30156102,  0.48439883, -0.18498607,  0.48837209,
        0.48837209, -0.5       , -1.        , -0.00917431, -1.        ,
       -1.        ,  0.90640259,  0.23297703, -0.58093009, -0.75305431,
       -1.        ])]
[1.64890034]


The next function is a random peptide selector, it generates 10 random numbers between 1 to 8000 and uses these numbers as indices to select peptides from the data and adds it to the training set.

In [57]:
def randPepSelector(APs, judp, modp, smiles, x_train_jd, x_train_md, Y_train):                           # a is APs, b is judp/modp
    for pep in range(10):                                                                                # generates random indecies, looks up those indecies data and adds to the training data
        i = random.randrange(1,8000)
        p = APs.iloc[i]
        print(p.name)
        Y_train = np.append(Y_train, np.array(p), axis=0)
        x_train_jd = np.append(x_train_jd, [np.array(judp.loc[p.name.encode('utf-8')])],axis=0)
        x_train_md = np.append(x_train_md, np.array(modp.iloc[smiles.index[smiles['Peptides']==p.name]]), axis=0)
    return Y_train,x_train_jd, x_train_md  

Next we call the random peptide selector and print the data to have a look at it.

In [58]:
Y_train,x_train_jd,x_train_md = randPepSelector(APs,judp,modp,smiles,x_train_jd,x_train_md,Y_train)                 # first we take a set of random peptides(10)
print(x_train_jd)
print(x_train_md)
print(Y_train)

GLN-ASP-SER
GLY-HSE-ASP
GLU-HSE-ALA
ASP-TYR-ALA
HSE-MET-ILE
GLU-ARG-LEU
ASP-SER-GLY
VAL-THR-LEU
LYS-ARG-SER
LYS-GLU-ARG
[[-1.         -1.         -0.7827501  -1.          0.29192555  0.
  -0.7237569  -1.         -0.11330056 -1.        ]
 [-0.8333333  -0.6666666  -0.17858994 -1.         -0.10973078 -0.33333334
  -0.03867412 -0.875      -0.07316196 -0.3333333 ]
 [-0.6666666  -1.         -0.28703415 -1.         -0.22567284 -0.33333334
  -0.23020267 -0.5        -0.32238638 -1.        ]
 [-0.6666666  -1.         -0.14225233 -1.          0.1076605  -0.33333334
  -0.02762437 -0.75        0.04214561 -1.        ]
 [-0.41666663 -1.         -0.08026218 -1.         -0.11801237 -0.33333334
   0.00552487 -0.4166667   0.1315453  -0.3333333 ]
 [-0.75       -1.          0.08568192 -0.3333333   0.768116    0.
   0.22651935 -0.90625     0.5011859  -1.        ]
 [-0.8333333  -0.3333333   0.17322206 -1.          0.09937891  0.
   0.421731   -0.9444444   0.42492247 -1.        ]
 [-0.9166667  -1.         -0.

In [59]:
def loc(s,Y_pred):                      #takes the APs of top peptides and returns their location in judred parameters so we can look up their names by their location
    y = []
    for i in range(len(s)):
        for j in range(len(Y_pred)):
            if s[i] == Y_pred[j]:
                if j not in y:
                    y.append(j)
    return y

The next function, takes in training data(judred parameters and AP scores) and trains a SVM. The hyperparameters for this model are given in the supporting information of the paper. Then it predicts the AP score for all peptides using the judred data. We then return the names of the top 1207 peptides which are predicted. The number 1207 comes from the formula given in the paper.   

In [60]:
# Block-1

def topN(n,judp,x_train_jd,Y_train):                                                            #takes the judred parameters and the training data 
    svr = SVR(kernel='rbf',gamma='scale',C=100,epsilon=0.1,max_iter=-1,tol=0.0001,verbose=0)    #trains a svm(rbf)
    svr.fit(x_train_jd,Y_train)
    x = []
    for i in range(8000):
        x.append(np.array(judp.iloc[i]))
    imputer = SimpleImputer(strategy='mean')                                                    #imputer removes NaN values from the data set
    x_imputed = imputer.fit_transform(x)
    Y_pred = svr.predict(x_imputed)                                                             #predicts AP scores for all peptides
    s = sorted(Y_pred)
    s = s[8000-n:]
    y_loc = loc(s,Y_pred)
    y_nam = ['']*len(y_loc)
    for i in range(len(y_loc)):
        y_nam[i] = judp.iloc[y_loc[i]].name
    return y_nam   

This function takes the names of the predicted peptides, looks the data up using the name and adds it the training data.

In [61]:
# Block-2

def addToTrain(y_nam,APs,judp,modp,x_train_jd,x_train_md,Y_train):                                              #takes the predicted peptide in previous iteration and adds them to training data
    for i in y_nam:
        if np.array(APs.loc[i.decode(encoding='utf-8')]) not in Y_train:
            x_train_jd = np.append(x_train_jd, [np.array(judp.loc[i])], axis=0)
            x_train_md = np.append(x_train_md, np.array(modp.iloc[smiles.index[smiles['Peptides']==i.decode(encoding='utf-8')]]), axis=0)
            Y_train = np.append(Y_train, np.array(APs.loc[i.decode(encoding='utf-8')]), axis=0)
    return x_train_jd,x_train_md,Y_train

This function trains a svm on the mordred data of the peptides in training dataset and gives the names of the predicted peptides.

In [62]:
# Block-3

def topn(n,smiles, modp, y_nam_jd, x_train_md, Y_train):
    svr = SVR(kernel='rbf',gamma='scale',C=100,epsilon=0.1,max_iter=-1,tol=0.0001,verbose=0)    #trains a svm(rbf)
    svr.fit(x_train_md,Y_train)
    x = np.empty((0,modp.shape[1]))
    for i in y_nam_jd:
        x = np.append(x, np.array(modp.iloc[smiles.index[smiles['Peptides']==i.decode(encoding='utf-8')]]), axis=0)
    Y_pred = svr.predict(x)                                                                     #predicts AP scores for all peptides
    pred = pd.DataFrame(data=Y_pred,index=y_nam_jd,columns=['prediction'])
    y_nam = np.array(pred['prediction'].nlargest(n=n).index)
    return y_nam


This function identifies if a peptide present in 'sap' the list of self assembling peptides.

In [63]:
def pepidentifier(y_nam,sap,a):
    for j in y_nam:
        b = j.decode(encoding='utf-8')
        for i in sap:            
            if b == i:
                a.append(b)
    return a

Next we run the active learning algorithm, using the functions defined above. In step 1, block 1 is called. In step 2, block 3 is called. In step 3, block 2 is called.

In [64]:
def runAL(num_iter,APs,judp,smiles,modp,x_train_jd,x_train_md,Y_train,sap):
    a = []                                                                                            #array to keep track if any self-assembling peptide is found
    for i in range(num_iter):                                                                         #a for loop is for the number of iterations as given in input
        y_nam_jd = topN(math.ceil((np.log(3)**2) * 1000),judp,x_train_jd,Y_train)                     #Step 1: we get the names of top 1207 peptides which are predicted by judred model by calling the function 'topN'
        y_nam= topn(10,smiles, modp, y_nam_jd, x_train_md, Y_train)                                   #Step 2: from the peptides predicted by judred model we predict the top 10 AP scoring peptides as predicted by the mordred model
        print(y_nam)
        a = pepidentifier(y_nam_jd,sap,a)                                                                #we call the peptide identifier function which checks if any self-assembling peptide is predicted.
        x_train_jd,x_train_md,Y_train = addToTrain(y_nam,APs,judp,modp,x_train_jd,x_train_md,Y_train) #Step 3: we add the top 10 peptides to the training set.
    b = []
    for j in range(len(a)):
        if a[j] not in b:
            b.append(a[j])
    print(b)
    return topn(40,smiles, modp, y_nam_jd, x_train_md, Y_train)                                       #after the last iteration we print the top 40 peptides predicted by the mordred model.

y_nam = runAL(20,APs,judp,smiles,modp,x_train_jd,x_train_md,Y_train,sap)                              #we run the active learning process for 80 iteration.
print(y_nam)

[b'HSE-ILE-MET' b'HSE-MET-ILE' b'ILE-HSE-MET' b'ILE-MET-HSE'
 b'MET-HSE-ILE' b'MET-ILE-HSE' b'HSE-VAL-MET' b'HSE-MET-VAL'
 b'MET-HSE-VAL' b'ILE-PHE-MET']
[b'TYR-MET-LEU' b'TYR-CYS-LEU' b'TYR-LEU-MET' b'TYR-LEU-CYS'
 b'VAL-GLY-MET' b'MET-GLY-LEU' b'HSE-GLY-MET' b'TYR-MET-CYS'
 b'TYR-CYS-MET' b'TYR-MET-MET']
[b'VAL-MET-HSE' b'TYR-MET-MET' b'HSE-MET-LEU' b'VAL-HSE-MET'
 b'TYR-MET-ILE' b'ASN-MET-PHE' b'TYR-ALA-MET' b'VAL-CYS-HSE'
 b'HSE-CYS-LEU' b'MET-HSE-LEU']
[b'TYR-MET-MET' b'TYR-MET-ILE' b'TYR-ALA-MET' b'HSE-MET-LEU'
 b'TYR-ILE-MET' b'HSE-CYS-LEU' b'VAL-MET-HSE' b'TYR-MET-VAL'
 b'TYR-MET-CYS' b'TYR-CYS-MET']
[b'TYR-MET-MET' b'TYR-MET-ILE' b'TYR-ALA-MET' b'TYR-MET-VAL'
 b'TYR-ILE-MET' b'HSE-MET-LEU' b'HSE-CYS-LEU' b'TYR-MET-ALA'
 b'TYR-CYS-MET' b'TYR-MET-CYS']
[b'TYR-MET-MET' b'TYR-MET-ILE' b'TYR-ALA-MET' b'TYR-MET-VAL'
 b'TYR-ILE-MET' b'HSE-MET-LEU' b'HSE-CYS-LEU' b'TYR-MET-ALA'
 b'TYR-CYS-MET' b'TYR-MET-CYS']
[b'TYR-MET-MET' b'TYR-MET-ILE' b'TYR-ALA-MET' b'TYR-MET-VAL'
 b'TYR-ILE-MET'

But we find that at max only one peptide is found in the process for 80 iterations, whereas 20 peptides where found within 15 iterations in the paper.  